# Meta-Learning V2 测试 Notebook

本notebook用于测试Meta-Learning V2方法，训练RNN成为"快速学习器"，
使其能够从1D规则快速适应并泛化到2D组合任务。

## 核心思想（V2版本）
- **Meta-Training**：训练RNN学习"如何快速学习"
  - **Support Set**: 1D规则样本（Sweet, Dry, Light, Full）
    - 使用规则向量 `[1,0,0,0]`, `[0,1,0,0]`, `[0,0,1,0]`, `[0,0,0,1]`
    - 每个规则生成多个wine pair样本
  - **Query Set**: 2D规则样本（组合规则）
    - 使用规则向量 `[1,0,1,0]`, `[1,0,0,1]`, `[0,1,1,0]`, `[0,1,0,1]`
    - 测试零样本泛化到2D组合规则
- **Meta-Testing**：测试1D→2D的泛化能力
- **输出格式**: 3类 [Wine1胜, Wine2胜, 平局]

## 🚀 Colab使用说明
1. **启用GPU**: Runtime -> Change runtime type -> GPU
2. **按顺序运行**: 从Cell 3开始，按顺序运行所有代码块
   - Cell 3: 克隆项目
   - Cell 4: 安装NumPy
   - Cell 5: 安装依赖并导入模块
   - 后续cells: 训练和测试

## 0. 在Colab中克隆项目（仅Colab需要）

**如果这是第一次在Colab中运行，请先克隆项目：**
- 从GitHub克隆项目：https://github.com/YifeiCAO/wine-spave
- 运行下面的cell自动克隆并设置项目路径


## 1. 环境设置和依赖安装


In [9]:
# 在Colab中克隆项目（仅Colab需要）
import os
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # 从GitHub克隆项目
    project_root = '/content/wine-spave'
    
    if not os.path.exists(project_root):
        print("📦 从GitHub克隆项目...")
        get_ipython().system('git clone https://github.com/YifeiCAO/wine-spave.git')
        print("✓ 项目克隆完成")
    else:
        print("✓ 项目已存在，跳过克隆")
    
    # 切换到项目目录
    os.chdir(project_root)
    print(f"✓ 项目根目录: {project_root}")
    print(f"✓ 当前工作目录: {os.getcwd()}")
else:
    # 本地环境
    current_dir = os.getcwd()
    if 'notebooks' in current_dir:
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir
    print(f"本地环境，项目根目录: {project_root}")


本地环境，项目根目录: /Users/yifei/Desktop/UCLA/wine-spave


In [10]:
# 修复NumPy版本兼容性（必须在导入torchvision之前）
import sys
import subprocess

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 安装NumPy 1.26.4（兼容PyTorch）...")
    # 清除numpy模块缓存（如果已导入）
    if 'numpy' in sys.modules:
        del sys.modules['numpy']
    
    # 直接重新安装NumPy 1.26.4
    try:
        get_ipython().system('pip install -q --force-reinstall numpy==1.26.4')
    except:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--force-reinstall", "numpy==1.26.4"])
    
    import numpy as np
    print(f"✓ NumPy已安装: {np.__version__}\n")


In [11]:
# 安装其他依赖并导入模块
import os
import sys
import random

IN_COLAB = 'google.colab' in sys.modules

# 安装依赖
if IN_COLAB:
    print("📦 安装依赖包...")
    try:
        get_ipython().system('pip install -q torch torchvision matplotlib tqdm')
    except:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision", "matplotlib", "tqdm"])
    print("✓ 依赖包安装完成\n")
else:
    # 本地环境：检查依赖
    try:
        import torch
    except ImportError:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision"])
    
    try:
        import matplotlib
    except ImportError:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "matplotlib"])
    
    try:
        import tqdm
    except ImportError:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tqdm"])

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 设置项目路径（在Colab中，Cell 3已经切换了工作目录）
if IN_COLAB:
    # 在Colab中，项目应该在/content/wine-spave（Cell 3已经切换到这里）
    project_root = '/content/wine-spave'
    # 确保在项目目录中
    if os.path.exists(project_root):
        os.chdir(project_root)
    else:
        print("⚠ 警告: 未找到wine-spave目录，请先运行Cell 3克隆项目")
        project_root = '/content'
else:
    # 本地环境
    current_dir = os.getcwd()
    if 'notebooks' in current_dir:
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir

# 添加项目路径到sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"项目根目录: {project_root}")
print(f"当前工作目录: {os.getcwd()}")

# GPU检测
if torch.cuda.is_available():
    device = "cuda"
    print(f"✓ GPU可用: {torch.cuda.get_device_name(0)}")
    print(f"  GPU数量: {torch.cuda.device_count()}")
    print(f"  显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    device = "cpu"
    print("⚠ GPU不可用，将使用CPU")
    if IN_COLAB:
        print("  提示: 在Colab中，请确保已启用GPU: Runtime -> Change runtime type -> GPU")

# 导入项目模块
print("\n导入项目模块...")
try:
    from data import GridDataGenerator
    from models import get_model
    from meta_learning_v2 import (
        meta_train_v2, 
        meta_test_v2
    )
    print("✓ 所有模块导入完成（使用Meta-Learning V2）")
except ImportError as e:
    print(f"❌ 导入失败: {e}")
    print(f"   请确保项目文件已正确上传到: {project_root}")
    print(f"   当前sys.path: {sys.path[:3]}")
    raise


项目根目录: /Users/yifei/Desktop/UCLA/wine-spave
当前工作目录: /Users/yifei/Desktop/UCLA/wine-spave/notebooks
⚠ GPU不可用，将使用CPU

导入项目模块...
✓ 所有模块导入完成（使用Meta-Learning V2）


In [12]:
# 基础参数类（V2版本）
class Args:
    def __init__(self):
        # 设备设置
        self.use_cuda = torch.cuda.is_available()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.seed = 42
        
        # 数据集设置
        self.use_images = False
        self.image_dir = 'images/faces16'
        self.training_regime = 'ungrouped'
        self.grid_size = 4
        self.ctx_order = 'first'
        self.inner_4x4 = False
        
        # 模型设置
        self.model_name = 'rnn'
        self.ctx_scale = 1.0
        self.measure_grad_norm = False
        
        # Meta-Learning V2参数
        self.meta_lr = 0.001
        # Support Set: 每个1D规则（Sweet, Dry, Light, Full）的样本数
        self.n_support_per_rule = 16  # 每个1D规则生成16个样本
        # Query Set: 2D规则样本总数
        self.n_query = 32  # 从所有2D规则样本中随机采样32个
        self.n_meta_iterations = 500  # 可以调整为≥10000用于正式训练
        self.n_tasks_per_batch = 4

# 创建参数对象
args = Args()

# 设置随机种子
torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

print("参数设置完成（V2版本）！")
print(f"设备: {args.device}")
print(f"模型: {args.model_name}")
print(f"Meta学习率: {args.meta_lr}")
print(f"Meta迭代次数: {args.n_meta_iterations}")
print(f"每个1D规则的Support样本数: {args.n_support_per_rule}")
print(f"Query Set大小: {args.n_query}")
print(f"输出类别数: 3 (Wine1胜, Wine2胜, 平局)")

参数设置完成（V2版本）！
设备: cpu
模型: rnn
Meta学习率: 0.001
Meta迭代次数: 500
每个1D规则的Support样本数: 16
Query Set大小: 32
输出类别数: 3 (Wine1胜, Wine2胜, 平局)


In [13]:
# 创建模型（V2版本不需要create_meta_learning_args）
model = get_model(args)
model = model.to(args.device)

print(f"✓ 模型创建完成（V2版本）")
print(f"  模型类型: {type(model).__name__}")
print(f"  设备: {args.device}")
print(f"  注意: 模型输出维度将在meta_train_v2中自动调整为3类")

Model is an LSTM
✓ 模型创建完成（V2版本）
  模型类型: RNN
  设备: cpu
  注意: 模型输出维度将在meta_train_v2中自动调整为3类


In [14]:
# 执行meta-training V2
meta_trained_model, meta_losses = meta_train_v2(
    model,
    args,
    n_meta_iterations=args.n_meta_iterations,
    n_tasks_per_batch=args.n_tasks_per_batch
)

修改模型输出维度为3类
开始Meta-Training V2...
  总迭代次数: 500
  每批任务数: 4
  每个1D规则的Support样本数: 16
  Query Set大小: 32
  输出类别数: 3 (Wine1胜, Wine2胜, 平局)



Meta-Training V2:  15%|▋    | 74/500 [01:32<08:53,  1.25s/iter, Loss=1.0338, Avg=1.0589, Min=0.9913]


KeyboardInterrupt: 

In [ ]:
# 可视化训练损失
plt.figure(figsize=(12, 4))

# 原始损失
plt.subplot(1, 2, 1)
plt.plot(meta_losses, alpha=0.6, linewidth=1)
plt.xlabel('Meta-Iteration')
plt.ylabel('Meta-Loss')
plt.title('Meta-Training Loss (Raw)')
plt.grid(True, alpha=0.3)

# 平滑损失
plt.subplot(1, 2, 2)
window = min(100, len(meta_losses) // 10)
if window > 1:
    moving_avg = np.convolve(meta_losses, np.ones(window)/window, mode='valid')
    plt.plot(meta_losses, alpha=0.3, label='Raw', linewidth=1)
    plt.plot(range(window-1, len(meta_losses)), moving_avg, 
             label=f'Moving Average (window={window})', linewidth=2)
else:
    plt.plot(meta_losses, alpha=0.6, linewidth=1)
plt.xlabel('Meta-Iteration')
plt.ylabel('Meta-Loss')
plt.title('Meta-Training Loss (Smoothed)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"初始Meta-Loss: {meta_losses[0]:.4f}")
print(f"最终Meta-Loss: {meta_losses[-1]:.4f}")
if len(meta_losses) > 100:
    print(f"最后100次平均: {np.mean(meta_losses[-100:]):.4f}")

In [ ]:
# 执行meta-testing V2（直接测试1D→2D泛化）
# V2版本不需要预先生成grid，会在内部自动生成
final_acc, accuracies, rule_results = meta_test_v2(
    meta_trained_model,
    args,
    n_test_tasks=20  # 测试20个任务
)

In [ ]:
# 结果总结（V2版本：3类输出）
print("="*60)
print("📊 Meta-Testing V2 结果总结")
print("="*60)
print(f"\n最终平均准确率: {final_acc:.4f} ({final_acc*100:.2f}%)")
print(f"测试任务数: {len(accuracies)}")
print(f"准确率范围: {min(accuracies):.4f} - {max(accuracies):.4f}")
print(f"\n基线比较（3类输出）:")
print(f"  随机猜测: 33.33% (3选1)")
print(f"  人类基线: ~66-80%")

# 按规则类型统计准确率
print("\n" + "="*60)
print("📊 各规则类型准确率统计")
print("="*60)

# 分离1D和2D规则
one_d_rules = ['Sweet', 'Dry', 'Light', 'Full']
two_d_rules = ['Sweet+Light', 'Sweet+Full', 'Dry+Light', 'Dry+Full']

print("\n【1D规则表现】")
one_d_accs = []
for rule_name in one_d_rules:
    if rule_name in rule_results:
        result = rule_results[rule_name]
        rule_acc = result['correct'] / result['total'] if result['total'] > 0 else 0.0
        one_d_accs.append(rule_acc)
        print(f"  {rule_name:12s}: {rule_acc:.4f} ({rule_acc*100:.2f}%) - {result['correct']}/{result['total']} 正确")
    else:
        print(f"  {rule_name:12s}: 无数据")

if one_d_accs:
    print(f"  1D规则平均: {np.mean(one_d_accs):.4f} ({np.mean(one_d_accs)*100:.2f}%)")

print("\n【2D规则表现】")
two_d_accs = []
for rule_name in two_d_rules:
    if rule_name in rule_results:
        result = rule_results[rule_name]
        rule_acc = result['correct'] / result['total'] if result['total'] > 0 else 0.0
        two_d_accs.append(rule_acc)
        print(f"  {rule_name:12s}: {rule_acc:.4f} ({rule_acc*100:.2f}%) - {result['correct']}/{result['total']} 正确")
    else:
        print(f"  {rule_name:12s}: 无数据")

if two_d_accs:
    print(f"  2D规则平均: {np.mean(two_d_accs):.4f} ({np.mean(two_d_accs)*100:.2f}%)")

print(f"\n【对比】")
if one_d_accs and two_d_accs:
    print(f"  1D规则平均准确率: {np.mean(one_d_accs):.4f} ({np.mean(one_d_accs)*100:.2f}%)")
    print(f"  2D规则平均准确率: {np.mean(two_d_accs):.4f} ({np.mean(two_d_accs)*100:.2f}%)")
    diff = np.mean(two_d_accs) - np.mean(one_d_accs)
    print(f"  差异: {diff:+.4f} ({diff*100:+.2f}%)")
    if diff > 0:
        print(f"  ✓ 2D规则表现更好（说明模型学会了组合）")
    elif diff < -0.1:
        print(f"  ⚠ 2D规则表现较差（可能需要更多训练）")
    else:
        print(f"  → 1D和2D规则表现相近")


# 可视化（2x3布局）
plt.figure(figsize=(16, 10))

# 1. 整体准确率分布
plt.subplot(2, 3, 1)
plt.hist(accuracies, bins=10, alpha=0.7, edgecolor='black')
plt.axvline(final_acc, color='r', linestyle='--', linewidth=2, label=f'平均: {final_acc:.3f}')
plt.axvline(0.3333, color='gray', linestyle='--', alpha=0.5, label='随机猜测 (33.33%)')
plt.axvline(0.66, color='green', linestyle='--', alpha=0.5, label='人类基线 (66%)')
plt.xlabel('准确率')
plt.ylabel('任务数量')
plt.title('整体准确率分布')
plt.legend()
plt.grid(True, alpha=0.3)

# 2. 准确率趋势
plt.subplot(2, 3, 2)
plt.plot(range(1, len(accuracies)+1), accuracies, 'o-', alpha=0.6, markersize=4)
plt.axhline(final_acc, color='r', linestyle='--', linewidth=2, label=f'平均: {final_acc:.3f}')
plt.axhline(0.3333, color='gray', linestyle='--', alpha=0.5, label='随机猜测 (33.33%)')
plt.axhline(0.66, color='green', linestyle='--', alpha=0.5, label='人类基线 (66%)')
plt.xlabel('任务序号')
plt.ylabel('准确率')
plt.title('准确率趋势')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. 各规则类型准确率对比（柱状图）
plt.subplot(2, 3, 3)
rule_names = []
rule_accs = []
rule_colors = []

for rule_name in one_d_rules + two_d_rules:
    if rule_name in rule_results:
        result = rule_results[rule_name]
        rule_acc = result['correct'] / result['total'] if result['total'] > 0 else 0.0
        rule_names.append(rule_name)
        rule_accs.append(rule_acc)
        # 1D规则用蓝色，2D规则用橙色
        rule_colors.append('steelblue' if rule_name in one_d_rules else 'orange')

if rule_names:
    bars = plt.bar(range(len(rule_names)), rule_accs, color=rule_colors, alpha=0.7, edgecolor='black')
    plt.axhline(0.3333, color='gray', linestyle='--', alpha=0.5, label='随机猜测')
    plt.axhline(0.66, color='green', linestyle='--', alpha=0.5, label='人类基线')
    plt.xticks(range(len(rule_names)), rule_names, rotation=45, ha='right')
    plt.ylabel('准确率')
    plt.title('各规则类型准确率')
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')
    # 添加数值标签
    for i, (bar, acc) in enumerate(zip(bars, rule_accs)):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{acc:.2f}', ha='center', va='bottom', fontsize=8)

# 4. 1D vs 2D规则对比
plt.subplot(2, 3, 4)
categories = ['1D规则', '2D规则']
if one_d_accs and two_d_accs:
    avg_accs = [np.mean(one_d_accs), np.mean(two_d_accs)]
    colors = ['steelblue', 'orange']
    bars = plt.bar(categories, avg_accs, color=colors, alpha=0.7, edgecolor='black')
    plt.axhline(0.3333, color='gray', linestyle='--', alpha=0.5, label='随机猜测')
    plt.axhline(0.66, color='green', linestyle='--', alpha=0.5, label='人类基线')
    plt.ylabel('平均准确率')
    plt.title('1D vs 2D 规则对比')
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')
    # 添加数值标签
    for bar, acc in zip(bars, avg_accs):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{acc:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# 5. 各规则类型的样本数量
plt.subplot(2, 3, 5)
rule_names_count = []
rule_counts = []
rule_colors_count = []

for rule_name in one_d_rules + two_d_rules:
    if rule_name in rule_results:
        rule_names_count.append(rule_name)
        rule_counts.append(rule_results[rule_name]['total'])
        rule_colors_count.append('steelblue' if rule_name in one_d_rules else 'orange')

if rule_names_count:
    bars = plt.bar(range(len(rule_names_count)), rule_counts, color=rule_colors_count, alpha=0.7, edgecolor='black')
    plt.xticks(range(len(rule_names_count)), rule_names_count, rotation=45, ha='right')
    plt.ylabel('样本数量')
    plt.title('各规则类型样本数量')
    plt.grid(True, alpha=0.3, axis='y')
    # 添加数值标签
    for bar, count in zip(bars, rule_counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(rule_counts)*0.01, 
                f'{count}', ha='center', va='bottom', fontsize=8)

# 6. 各规则类型的正确/错误分布
plt.subplot(2, 3, 6)
rule_names_pie = []
correct_counts = []
wrong_counts = []

for rule_name in one_d_rules + two_d_rules:
    if rule_name in rule_results:
        result = rule_results[rule_name]
        rule_names_pie.append(rule_name)
        correct_counts.append(result['correct'])
        wrong_counts.append(result['total'] - result['correct'])

if rule_names_pie:
    x = np.arange(len(rule_names_pie))
    width = 0.35
    plt.bar(x - width/2, correct_counts, width, label='正确', color='green', alpha=0.7)
    plt.bar(x + width/2, wrong_counts, width, label='错误', color='red', alpha=0.7)
    plt.xticks(x, rule_names_pie, rotation=45, ha='right')
    plt.ylabel('样本数量')
    plt.title('各规则类型正确/错误分布')
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# 评估结果
print(f"\n评估:")
if final_acc > 0.66:
    print(f"  ✓ Meta-Learning V2成功！准确率({final_acc*100:.2f}%)超过人类基线下限")
elif final_acc > 0.3333:
    print(f"  ⚠ Meta-Learning V2部分成功，准确率({final_acc*100:.2f}%)超过随机但未达到人类基线")
else:
    print(f"  ✗ Meta-Learning V2需要进一步优化，准确率({final_acc*100:.2f}%)接近随机水平")